In [436]:
# Himani Parikh
# 10/08/2022
# Class Assignment 3
#Produce sparse shingles binary matrix and then create the the minHash Signature for the 3 documents Using Hash Functions or Using 3 permutations of Pi.

# Revision History
# scl   10/06/2022    Initial Version
# scl   10/15/2022    Refactor 

In [437]:
import numpy as np
import pandas as pd
import random

In [438]:
s1 ="the plane was ready for touch down"
s2 ="the quarterback scored a touchdown"
s3 ="makes you want to move your dancing feet"
k_shingle = 5 
corpus = [s1, s2, s3]

In [439]:
# 1) Shingling: Convert documents to sets

def f_shingles(string, k):
  shingle = []
  for i in range(len(string) - k + 1):
    shingle.append(string[i:i+k])
  return set(shingle)

In [440]:
def f_cleanDocument(d):
  doc = d
  from string import digits, punctuation
  doc = doc.lower()
  doc = doc.replace('\n', ' ')
  doc = doc.translate(str.maketrans('', '',punctuation))
  remove_digits = str.maketrans('', '', digits)
  doc = doc.translate(remove_digits)
  return doc

In [441]:
def f_calcTermFrequency(df, t, id):
  c = []
  t = list(t)
  for word in df['shingle'].values:
    c.append(t.count(word))
  df['shingle'+'_'+id] = c
  return df

In [442]:
def f_matrix(text, elements):
  matrix = []
  for x in elements:
    if x in text:
      matrix.append(1)
    else:
      matrix.append(0)
  return matrix

In [443]:
def f_minhash(df):
  h1_list = []
  h2_list = []
  h3_list = []
  
  for x in range (len(df)):
     h1_list.append(((5*x) + 2) % len(df))
     h2_list.append(((3*x) + 1) % len(df))
     h3_list.append(((2*x) + 4) % len(df))
    
  df['h1'] = h1_list 
  df['h2'] = h2_list
  df['h3'] = h3_list
  return df

In [444]:
def f_signatures(df,df_sig):
  print(df_sig.head())
  print(df.head())
  for i in df.index:
    for j in range(1, 4):
      if df.loc[i,'shingle_'+str(j)] > 0:
        for p in range(1, 4):
          if df_sig.loc['h'+str(p),'s'+str(j)] > df.loc[i,'h'+str(p)]:
            df_sig.loc['h'+str(p),'s'+str(j)] = df.loc[i,'h'+str(p)]
  return df_sig

In [445]:
def main():
  universal_set = set()
  # first pass
  for document in corpus:
    shingle_set = f_shingles(document, k_shingle)
    universal_set.update(shingle_set)
  df_universal = pd.DataFrame(universal_set,columns=['shingle'])

  # second pass
  doc_id = 0
  for document in corpus:
    doc_id      += 1
    clean_doc   = f_cleanDocument(document)
    shingle_set = f_shingles(clean_doc, k_shingle)
    tfreq  = f_calcTermFrequency(df_universal,shingle_set,str(doc_id))
  
  # # Next minHash but permutation with basic hash functions h1 and h2
  df_minHash = f_minhash(df_universal)

  # # Next compute the minHash Signature
  N = len(corpus) # N Documents
  h = 3 # h hashs functions for permutations
  np_signature  = np.zeros((h, N), dtype=int)
  df_signature  = pd.DataFrame(np_signature,columns=['s1','s2','s3'], index=['h1','h2','h3'])
  df_signature  = df_signature.replace(0,99999)
  df_minHashSig = f_signatures(df_minHash, df_signature)

  return df_universal, df_minHash, df_minHashSig

In [446]:
universal, minHash, minHashSig = main()
minHashSig

       s1     s2     s3
h1  99999  99999  99999
h2  99999  99999  99999
h3  99999  99999  99999
  shingle  shingle_1  shingle_2  shingle_3  h1  h2  h3
0    plan          1          0          0   2   1   4
1   o mov          0          0          1   7   4   6
2   k sco          0          1          0  12   7   8
3    scor          0          1          0  17  10  10
4   e was          1          0          0  22  13  12


,s1,s2,s3
h1,2,0,5
h2,0,2,4
h3,2,4,0
